# Stock Data Processing Documentation

## Overview

This Python module processes and integrates multiple data sources for stock market analysis, combining price data with sentiment analysis from various sources (Reddit, news, MD&A reports) and financial metrics (ROE, PE ratios). The code creates several dataset variants for comparative machine learning model evaluation, with the goal of predicting stock returns.

## Installation and Dependencies

The code requires the following Python packages:
- pandas
- numpy
- os
- logging
- datetime
- json

## Data Sources

The module processes data from five main sources:

1. **Stock Price Data**: Historical price information including open, high, low, close, volume, and adjusted close
2. **Reddit Sentiment**: Aggregated sentiment analysis from Reddit posts
3. **News Sentiment**: Sentiment analysis from financial news articles
4. **MD&A Sentiment**: Sentiment analysis from Management Discussion & Analysis sections of quarterly reports
5. **ROE/PE Data**: Return on Equity and Price-to-Earnings ratio financial metrics

## File Structure

Input data files should be organized as follows:
```
data/
├── stock_price_data/top30/          # Price data CSV files
│   └── TICKER_prices.csv
├── pe_roe_timeseries/               # PE and ROE data
│   └── TICKER_2023_24.csv
sentiment_results/
├── mda/top30/                       # MD&A sentiment analysis
│   └── TICKER_2023_24.csv
├── news/top30/                      # News sentiment analysis
│   └── news_sentiment_TICKER_with_vader_finbert.csv
└── reddit/                          # Reddit sentiment analysis
    └── reddit_sentiment_YEAR_v3.csv
```

Output processed data will be saved to:
```
processed_data/
└── TICKER_VARIANT.csv
```

## Main Functions

### Data Loading Functions

#### `load_price_data(ticker)`
Loads price data from CSV files and calculates log returns and next-day return (target variable).

#### `load_reddit_sentiment(year_list=['2023', '2024'])`
Loads and aggregates Reddit sentiment data with volume weighting to emphasize days with higher post activity.

#### `load_news_sentiment(ticker)`
Loads company-specific news sentiment data with volume weighting to emphasize days with more news coverage.

#### `load_mda_sentiment(ticker, alpha=0.2)`
Loads MD&A sentiment data with dynamic exponential decay weighting. For each daily data point, calculates a weighted average of all previous quarterly reports where weight decays exponentially with time.

#### `load_roe_pe_data(ticker, alpha=0.2)`
Loads and processes ROE/PE data with similar dynamic exponential decay weighting.

### Data Processing Functions

#### `merge_datasets(price_df, reddit_df=None, news_df=None, mda_df=None, roe_pe_df=None)`
Merges all datasets on the Date column, starting with price data as the base.

#### `process_ticker(ticker, alpha_mda=0.2, alpha_roe_pe=0.2, reddit_years=['2023', '2024'])`
Orchestrates the complete processing pipeline for a single ticker:
1. Loads all datasets
2. Merges datasets
3. Filters to include only data from 2023-01-01 to 2024-12-31
4. Handles missing values and performs final preprocessing

#### `create_dataset_variants(merged_data)`
Creates four different variants of the dataset for model comparison:
1. Price data only
2. Price + PE/ROE data only
3. Price + Sentiment data only
4. All data combined

#### `save_processed_data(dataset_variants, ticker, output_dir='processed_data')`
Saves all dataset variants to CSV files.

### Main Execution Function

#### `main()`
Controls the overall execution flow:
1. Processes a list of 30 top stock tickers
2. Handles special cases (e.g., BRK.B/BRK-B format variations)
3. Tracks successful and failed ticker processing
4. Creates a summary report of processing results

## Features and Techniques

### Sentiment Analysis Integration
- **Volume Weighting**: Applies log(1+count) weighting to sentiment scores based on post/article volume
- **Exponential Decay**: Implements dynamic time-based weighting for quarterly reports (MD&A, ROE/PE)
- **Multiple Sentiment Models**: Integrates various sentiment indicators (VADER, FinBERT, custom models)

### Data Preprocessing
- **Log Returns**: Calculates logarithmic returns as the primary financial measurement
- **Target Variable**: Creates next_day_return as the prediction target
- **Missing Value Handling**: Robust handling of NaN values with appropriate methods

## Output Datasets

The module creates four dataset variants for each ticker:
1. **price_only**: Contains only price-related variables
2. **price_pe_roe**: Price data plus PE ratio and ROE metrics
3. **price_sentiment**: Price data plus all sentiment indicators
4. **all_data**: Complete dataset with all variables

## Usage

Run the module directly:
```python
python stock_data_processing.py
```

The script will:
1. Process all 30 tickers defined in the main function
2. Output processed datasets to the 'processed_data' directory
3. Generate a processing_summary.json with results

## Error Handling

The code includes comprehensive error handling and logging:
- Each function logs detailed information about its execution
- Processing failures for individual tickers are gracefully handled
- A summary of successful and failed tickers is generated

## Customization

Key parameters that can be customized:
- `alpha_mda` and `alpha_roe_pe`: Control decay rate for exponential weighting
- `reddit_years`: List of years to include for Reddit sentiment data
- Ticker list in the main function can be modified to process different stocks

In [1]:
import pandas as pd
import numpy as np
import os
import logging
from datetime import datetime
import json

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("stock_data_processing")

# Define file paths according to user specifications
PRICE_PATH = 'data/stock_price_data/top30/'
ROE_PE_PATH = 'data/pe_roe_timeseries/'
MDA_PATH = 'sentiment_results/mda/top30/'
NEWS_PATH = 'sentiment_results/news/top30/'
REDDIT_PATH = 'sentiment_results/reddit/'

def load_price_data(ticker):
    """
    Load price data and calculate log returns
    """
    file_path = os.path.join(PRICE_PATH, f"{ticker}_prices.csv")
    logger.info(f"Loading price data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"Price data file not found: {file_path}")
        return None
    
    df = pd.read_csv(file_path)
    
    # Ensure Date column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Sort by date
    df = df.sort_values('Date')
    
    # Calculate log returns
    df['log_return'] = np.log(df['Adjusted_close'] / df['Adjusted_close'].shift(1))
    
    # Add target variable (next day's return)
    df['next_day_return'] = df['log_return'].shift(-1)
    
    logger.info(f"Loaded price data: {df.shape[0]} rows, {df.shape[1]} columns")
    return df


def load_reddit_sentiment(year_list=['2023', '2024']):
    """
    Load and aggregate Reddit sentiment data with volume weighting
    """
    all_data = []
    logger.info(f"Loading Reddit sentiment data for years: {year_list}")
    
    for year in year_list:
        file_path = os.path.join(REDDIT_PATH, f"reddit_sentiment_{year}_v3.csv")
        if os.path.exists(file_path):
            logger.info(f"Loading Reddit data from {file_path}")
            df = pd.read_csv(file_path)
            all_data.append(df)
        else:
            logger.warning(f"Reddit data file not found: {file_path}")
    
    if not all_data:
        logger.error("No Reddit data files found")
        return None
    
    # Concatenate all years of data
    reddit_data = pd.concat(all_data, ignore_index=True)
    
    # Select only the specified columns
    specified_columns = [
        'subreddit', 'date', 
        'polarity_detection', 'emotion_detection', 'intent_analysis', 'subjectivity_objectivity',
        'fine_grained_sentiment_avg', 'aspect_based_sentiment_avg', 'topic_sentiment_analysis_avg', 'contextual_sentiment_avg',
        'gemini_overall_sentiment', 'gemini_emotional_sentiment', 'gemini_contextual_sentiment',
        'vader_compound', 'vader_pos', 'vader_neg', 'vader_neu',
        'finbert_positive', 'finbert_negative', 'finbert_neutral', 'finbert_sentiment'
    ]
    
    available_columns = list(set(specified_columns) & set(reddit_data.columns))
    reddit_data = reddit_data[available_columns]
    
    # Convert date to datetime
    reddit_data['date'] = pd.to_datetime(reddit_data['date'])
    
    # Calculate post count per day
    daily_post_count = reddit_data.groupby('date').size().reset_index(name='post_count')
    
    # Calculate mean sentiment for each specified sentiment column
    sentiment_columns = [col for col in available_columns if col not in ['subreddit', 'date']]
    
    # Initialize an empty DataFrame to store results
    daily_sentiment = daily_post_count[['date', 'post_count']].copy()
    
    # Calculate log-weighted sentiment for each column
    for col in sentiment_columns:
        # Calculate mean sentiment per day
        mean_sentiment = reddit_data.groupby('date')[col].mean().reset_index(name=f'mean_{col}')
        
        # Merge with post count
        daily_sentiment = pd.merge(daily_sentiment, mean_sentiment, on='date', how='left')
        
        # Calculate volume-weighted sentiment: Mean Sentiment × log(1+Post Count)
        daily_sentiment[f'reddit_{col}'] = daily_sentiment[f'mean_{col}'] * np.log1p(daily_sentiment['post_count'])
        
        # Drop the mean column to keep the DataFrame clean
        daily_sentiment = daily_sentiment.drop(f'mean_{col}', axis=1)
    
    # Rename date column for consistency
    daily_sentiment = daily_sentiment.rename(columns={'date': 'Date'})
    
    logger.info(f"Processed Reddit sentiment data: {daily_sentiment.shape[0]} rows, {daily_sentiment.shape[1]} columns")
    return daily_sentiment


def load_news_sentiment(ticker):
    """
    Load and aggregate news sentiment data with volume weighting
    """
    file_path = os.path.join(NEWS_PATH, f"news_sentiment_{ticker}_with_vader_finbert.csv")
    logger.info(f"Loading news sentiment data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"News sentiment file not found: {file_path}")
        return None
    
    news_data = pd.read_csv(file_path)
    
    # Select only the specified columns
    specified_columns = [
        'ticker', 'date', 'title',  # Including title to count articles
        'polarity_detection', 'emotion_detection', 'intent_analysis', 'subjectivity_objectivity',
        'fine_grained_sentiment_avg', 'aspect_based_sentiment_avg', 'topic_sentiment_analysis_avg', 'contextual_sentiment_avg',
        'gemini_overall_sentiment', 'gemini_emotional_sentiment', 'gemini_contextual_sentiment',
        'vader_compound', 'vader_pos', 'vader_neg', 'vader_neu',
        'finbert_positive', 'finbert_negative', 'finbert_neutral', 'finbert_sentiment'
    ]
    
    available_columns = list(set(specified_columns) & set(news_data.columns))
    news_data = news_data[available_columns]
    
    # Convert date to datetime
    news_data['date'] = pd.to_datetime(news_data['date'])
    
    # Calculate article count per day
    daily_article_count = news_data.groupby('date').size().reset_index(name='article_count')
    
    # Calculate mean sentiment for each specified sentiment column
    sentiment_columns = [col for col in available_columns if col not in ['ticker', 'date', 'title']]
    
    # Initialize an empty DataFrame to store results
    daily_sentiment = daily_article_count[['date', 'article_count']].copy()
    
    # Calculate log-weighted sentiment for each column
    for col in sentiment_columns:
        # Calculate mean sentiment per day
        mean_sentiment = news_data.groupby('date')[col].mean().reset_index(name=f'mean_{col}')
        
        # Merge with article count
        daily_sentiment = pd.merge(daily_sentiment, mean_sentiment, on='date', how='left')
        
        # Calculate volume-weighted sentiment: Mean Sentiment × log(1+Article Count)
        daily_sentiment[f'news_{col}'] = daily_sentiment[f'mean_{col}'] * np.log1p(daily_sentiment['article_count'])
        
        # Drop the mean column to keep the DataFrame clean
        daily_sentiment = daily_sentiment.drop(f'mean_{col}', axis=1)
    
    # Rename date column for consistency
    daily_sentiment = daily_sentiment.rename(columns={'date': 'Date'})
    
    logger.info(f"Processed news sentiment data: {daily_sentiment.shape[0]} rows, {daily_sentiment.shape[1]} columns")
    return daily_sentiment


def load_mda_sentiment(ticker, alpha=0.2):
    """
    Load MDA sentiment data with dynamic exponential decay weighting
    
    For each daily data point, we calculate a weighted average of all previous
    quarterly reports, where the weight decays exponentially with the time distance.
    """
    file_path = os.path.join(MDA_PATH, f"{ticker}_2023_24.csv")
    logger.info(f"Loading MDA sentiment data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"MDA sentiment file not found: {file_path}")
        return None
    
    mda_data = pd.read_csv(file_path)
    
    # Select only the specified columns
    specified_columns = [
        'Date',
        'polarity_detection', 'emotion_detection', 'intent_analysis', 'subjectivity_objectivity',
        'gemini_overall_sentiment', 'gemini_emotional_sentiment', 'gemini_contextual_sentiment',
        'fine_grained_sentiment_avg', 'aspect_based_sentiment_avg', 'topic_sentiment_analysis_avg', 'contextual_sentiment_avg',
        'vader_compound', 'vader_pos', 'vader_neg', 'vader_neu',
        'finbert_positive', 'finbert_negative', 'finbert_neutral', 'finbert_sentiment'
    ]
    
    available_columns = list(set(specified_columns) & set(mda_data.columns))
    mda_data = mda_data[available_columns]
    
    # Convert date to datetime
    mda_data['Date'] = pd.to_datetime(mda_data['Date'])
    
    # Sort by date
    mda_data = mda_data.sort_values('Date')
    
    # Get sentiment columns
    sentiment_columns = [col for col in available_columns if col != 'Date']
    
    # Create a date range that spans the entire period (daily)
    start_date = mda_data['Date'].min()
    end_date = mda_data['Date'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create a DataFrame with all dates
    daily_df = pd.DataFrame({'Date': all_dates})
    
    # Calculate lambda from alpha
    # Relationship: α ≈ 1 - e^(-λ)
    # So λ ≈ -ln(1-α)
    lambda_param = -np.log(1 - alpha)
    logger.info(f"Using decay parameter lambda={lambda_param:.4f} (from alpha={alpha})")
    
    # For each sentiment column, calculate weighted values for all days
    weighted_columns = {}
    
    for col in sentiment_columns:
        # Create a Series for this column
        col_values = pd.Series(index=mda_data['Date'], data=mda_data[col].values)
        
        # Check variation in original data for this column
        unique_values = col_values.unique()
        value_range = col_values.max() - col_values.min()
        
        # Log information about this column's values
        logger.info(f"Column {col}: {len(unique_values)} unique values, range: {value_range}")
        
        # If this column has minimal variation, potentially adjust lambda to increase decay visibility
        adjusted_lambda = lambda_param
        if len(unique_values) < 3 or value_range < 0.1:
            # Double the decay rate for columns with little variation
            adjusted_lambda = lambda_param * 2
            logger.info(f"Adjusted lambda for {col} to {adjusted_lambda} due to low variation")
        
        # For each day, calculate weighted average of all previous MDA reports
        weighted_values = []
        
        for day in all_dates:
            # Get all MDA reports up to this day
            past_reports = col_values[col_values.index <= day]
            
            if past_reports.empty:
                weighted_values.append(np.nan)
                continue
            
            # Calculate days since each report
            days_since = [(day - report_date).days for report_date in past_reports.index]
            
            # Calculate weights using continuous exponential decay with adjusted lambda
            weights = np.exp(-adjusted_lambda * np.array(days_since))
            
            # Normalize weights to sum to 1
            weights = weights / weights.sum()
            
            # Calculate weighted average
            weighted_avg = np.sum(past_reports.values * weights)
            weighted_values.append(weighted_avg)
        
        # Add to dictionary of weighted columns
        weighted_columns[f'mda_{col}'] = weighted_values
    
    # Add weighted columns to daily DataFrame
    for col_name, values in weighted_columns.items():
        daily_df[col_name] = values
    
    logger.info(f"Processed MDA sentiment data with dynamic decay: {daily_df.shape[0]} rows, {daily_df.shape[1]} columns")
    return daily_df


def load_roe_pe_data(ticker, alpha=0.2):
    """
    Load and process ROE/PE data with dynamic exponential decay weighting
    
    For each daily data point, we calculate a weighted average of all previous
    quarterly reports, where the weight decays exponentially with the time distance.
    """
    file_path = os.path.join(ROE_PE_PATH, f"{ticker}_2023_24.csv")
    logger.info(f"Loading ROE/PE data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"ROE/PE file not found: {file_path}")
        return None
    
    roe_pe_data = pd.read_csv(file_path)
    
    # Convert date to datetime
    roe_pe_data['Date'] = pd.to_datetime(roe_pe_data['Date'])
    
    # Sort by date
    roe_pe_data = roe_pe_data.sort_values('Date')
    
    # Convert ROE from string percentage to float if needed
    if roe_pe_data['ROE'].dtype == 'object':
        roe_pe_data['ROE'] = roe_pe_data['ROE'].str.replace('%', '').astype(float) / 100
    
    # Create a date range that spans the entire period (daily)
    start_date = roe_pe_data['Date'].min()
    end_date = roe_pe_data['Date'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create a DataFrame with all dates
    daily_df = pd.DataFrame({'Date': all_dates})
    
    # Calculate lambda from alpha
    # Relationship: α ≈ 1 - e^(-λ)
    # So λ ≈ -ln(1-α)
    lambda_param = -np.log(1 - alpha)
    logger.info(f"Using decay parameter lambda={lambda_param:.4f} (from alpha={alpha})")
    
    # For each financial metric, calculate weighted values for all days
    weighted_columns = {}
    original_columns = {}
    
    for col in ['PE_Ratio', 'ROE']:
        # Create a Series for this column
        col_values = pd.Series(index=roe_pe_data['Date'], data=roe_pe_data[col].values)
        
        # Keep track of original values for each date (for comparison)
        original_series = pd.Series(index=roe_pe_data['Date'], data=roe_pe_data[col].values)
        original_df = pd.DataFrame({'Date': original_series.index, col: original_series.values})
        
        # For each day, calculate weighted average of all previous financial reports
        weighted_values = []
        
        for day in all_dates:
            # Get all financial reports up to this day
            past_reports = col_values[col_values.index <= day]
            
            if past_reports.empty:
                weighted_values.append(np.nan)
                continue
            
            # Calculate days since each report
            days_since = [(day - report_date).days for report_date in past_reports.index]
            
            # Calculate weights using continuous exponential decay
            weights = np.exp(-lambda_param * np.array(days_since))
            
            # Normalize weights to sum to 1
            weights = weights / weights.sum()
            
            # Calculate weighted average
            weighted_avg = np.sum(past_reports.values * weights)
            weighted_values.append(weighted_avg)
        
        # Add to dictionaries
        weighted_columns[f'weighted_{col}'] = weighted_values
        
        # Merge original values
        daily_df = pd.merge(daily_df, original_df, on='Date', how='left')
    
    # Add weighted columns to daily DataFrame
    for col_name, values in weighted_columns.items():
        daily_df[col_name] = values
    
    # Forward fill missing original values
    daily_df = daily_df.fillna(method='ffill')
    
    logger.info(f"Processed ROE/PE data with dynamic decay: {daily_df.shape[0]} rows, {daily_df.shape[1]} columns")
    return daily_df


def merge_datasets(price_df, reddit_df=None, news_df=None, mda_df=None, roe_pe_df=None):
    """
    Merge all datasets on Date
    """
    # Start with price data as the base
    merged_df = price_df.copy()
    logger.info(f"Starting merge with price data: {merged_df.shape[0]} rows")
    
    # Merge Reddit sentiment
    if reddit_df is not None:
        merged_df = pd.merge(merged_df, reddit_df, on='Date', how='left')
        logger.info(f"After merging Reddit data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    # Merge News sentiment
    if news_df is not None:
        merged_df = pd.merge(merged_df, news_df, on='Date', how='left')
        logger.info(f"After merging News data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    # Merge MDA with dynamic decay weighting
    if mda_df is not None:
        # With the new dynamic decay approach, we now have daily values
        # so we can do a simple left join without forward fill
        merged_df = pd.merge(merged_df, mda_df, on='Date', how='left')
        logger.info(f"After merging MDA data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    # Merge ROE/PE with dynamic decay weighting
    if roe_pe_df is not None:
        # With the new dynamic decay approach, we now have daily values
        # so we can do a simple left join without forward fill
        merged_df = pd.merge(merged_df, roe_pe_df, on='Date', how='left')
        logger.info(f"After merging ROE/PE data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    return merged_df


def process_ticker(ticker, alpha_mda=0.2, alpha_roe_pe=0.2, reddit_years=['2023', '2024']):
    """
    Process all data for a single ticker
    """
    logger.info(f"Starting data processing for {ticker}")
    
    # 1. Load all datasets
    price_data = load_price_data(ticker)
    if price_data is None:
        logger.error(f"Failed to load price data for {ticker}")
        return None
    
    reddit_data = load_reddit_sentiment(reddit_years)
    news_data = load_news_sentiment(ticker)
    mda_data = load_mda_sentiment(ticker, alpha=alpha_mda)
    roe_pe_data = load_roe_pe_data(ticker, alpha=alpha_roe_pe)
    
    # 2. Merge datasets
    merged_data = merge_datasets(price_data, reddit_data, news_data, mda_data, roe_pe_data)
    
    # 3. Filter to only include data from 2023-01-01 to 2024-12-31
    start_date = pd.Timestamp('2023-01-01')
    end_date = pd.Timestamp('2024-12-31')
    merged_data = merged_data[(merged_data['Date'] >= start_date) & (merged_data['Date'] <= end_date)]
    logger.info(f"Filtered data to date range: {start_date} to {end_date}")
    
    # 4. Final preprocessing
    # Replace infinities with NaN
    merged_data = merged_data.replace([np.inf, -np.inf], np.nan)
    
    # Drop rows with NaN in log_return or next_day_return
    # These are essential as they are the target variables
    merged_data = merged_data.dropna(subset=['log_return', 'next_day_return'])
    
    # For other columns, fill NaNs with column means
    cols_to_fill = [col for col in merged_data.columns if col not in ['Date', 'log_return', 'next_day_return']]
    merged_data[cols_to_fill] = merged_data[cols_to_fill].fillna(merged_data[cols_to_fill].mean())
    
    logger.info(f"Final dataset for {ticker} has shape: {merged_data.shape}")
    
    # Create a column with the ticker name for reference
    merged_data['ticker'] = ticker
    
    return merged_data


def create_dataset_variants(merged_data):
    """
    Create different variants of the dataset for model comparison:
    1. Price data only
    2. Price + PE/ROE data only
    3. Price + Sentiment data only
    4. All data
    """
    # Price data only (keep necessary columns)
    price_cols = ['Date', 'ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'log_return', 'next_day_return']
    price_only = merged_data[price_cols].copy()
    
    # Price + PE/ROE
    pe_roe_cols = [col for col in merged_data.columns if any(term in col for term in ['PE_Ratio', 'ROE'])]
    price_pe_roe = merged_data[price_cols + pe_roe_cols].copy()
    
    # Price + Sentiment
    sentiment_cols = [col for col in merged_data.columns if any(
        prefix in col for prefix in ['reddit_', 'news_', 'mda_'])]
    price_sentiment = merged_data[price_cols + sentiment_cols].copy()
    
    # All data is already in merged_data
    
    return {
        'price_only': price_only,
        'price_pe_roe': price_pe_roe,
        'price_sentiment': price_sentiment,
        'all_data': merged_data
    }


def save_processed_data(dataset_variants, ticker, output_dir='processed_data'):
    """
    Save all dataset variants to files
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save each variant
    for variant_name, data in dataset_variants.items():
        file_path = os.path.join(output_dir, f"{ticker}_{variant_name}.csv")
        data.to_csv(file_path, index=False)
        logger.info(f"Saved {variant_name} dataset to {file_path}")


def main():
    """
    Main function to process multiple tickers
    """
    # List of tickers to process
    tickers = [
        "AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "META", "BRK.B", "AVGO", "TSLA", "LLY", 
        "WMT", "JPM", "V", "MA", "XOM", "COST", "UNH", "HD", "PG", "JNJ",
        "ABBV", "CRM", "BAC", "ORCL", "MRK", "CVX", "WFC", "KO", "CSCO", "ACN"
    ]
    
    # Create a directory for results
    os.makedirs('processed_data', exist_ok=True)
    
    # Track successful and failed tickers
    successful_tickers = []
    failed_tickers = []
    
    # Process each ticker
    for ticker in tickers:
        logger.info(f"Processing ticker: {ticker}")
        
        try:
            # Process the ticker
            merged_data = process_ticker(ticker)
            
            # Handle the BRK.B / BRK-B special case
            if merged_data is None and ticker == "BRK.B":
                logger.info(f"Failed to process {ticker}, trying alternative format BRK-B")
                merged_data = process_ticker("BRK-B")
                
                # If successful with the alternative format, use BRK-B going forward
                if merged_data is not None:
                    ticker = "BRK-B"
            elif merged_data is None and ticker == "BRK-B":
                logger.info(f"Failed to process {ticker}, trying alternative format BRK.B")
                merged_data = process_ticker("BRK.B")
                
                # If successful with the alternative format, use BRK.B going forward
                if merged_data is not None:
                    ticker = "BRK.B"
                    
            # If data processing was successful
            if merged_data is not None:
                # Create dataset variants
                dataset_variants = create_dataset_variants(merged_data)
                
                # Save processed data
                save_processed_data(dataset_variants, ticker)
                
                logger.info(f"Completed processing for {ticker}")
                successful_tickers.append(ticker)
            else:
                logger.error(f"Failed to process {ticker} after all attempts")
                failed_tickers.append(ticker)
                
        except Exception as e:
            logger.error(f"Error processing {ticker}: {str(e)}")
            failed_tickers.append(ticker)
    
    # Print summary
    logger.info("=" * 50)
    logger.info("Processing Summary:")
    logger.info(f"Successfully processed {len(successful_tickers)} tickers: {', '.join(successful_tickers)}")
    logger.info(f"Failed to process {len(failed_tickers)} tickers: {', '.join(failed_tickers)}")
    
    # Create summary file
    summary = {
        'successful_tickers': successful_tickers,
        'failed_tickers': failed_tickers,
        'processing_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open('processing_summary.json', 'w') as f:
        json.dump(summary, f, indent=4)
    
    logger.info("Processing complete. Summary saved to processing_summary.json")


if __name__ == "__main__":
    main()

2025-04-27 16:54:11,960 - stock_data_processing - INFO - Processing ticker: AAPL
2025-04-27 16:54:11,961 - stock_data_processing - INFO - Starting data processing for AAPL
2025-04-27 16:54:11,962 - stock_data_processing - INFO - Loading price data from data/stock_price_data/top30/AAPL_prices.csv
2025-04-27 16:54:11,969 - stock_data_processing - INFO - Loaded price data: 1561 rows, 9 columns
2025-04-27 16:54:11,969 - stock_data_processing - INFO - Loading Reddit sentiment data for years: ['2023', '2024']
2025-04-27 16:54:11,970 - stock_data_processing - INFO - Loading Reddit data from sentiment_results/reddit/reddit_sentiment_2023_v3.csv
2025-04-27 16:54:12,069 - stock_data_processing - INFO - Loading Reddit data from sentiment_results/reddit/reddit_sentiment_2024_v3.csv
2025-04-27 16:54:12,243 - stock_data_processing - INFO - Processed Reddit sentiment data: 704 rows, 21 columns
2025-04-27 16:54:12,246 - stock_data_processing - INFO - Loading news sentiment data from sentiment_results/

# Archive

In [1]:
import pandas as pd
import numpy as np
import os
import logging
from datetime import datetime

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("stock_data_processing")

# Define file paths according to user specifications
PRICE_PATH = 'data/stock_price_data/top30/'
ROE_PE_PATH = 'data/pe_roe_timeseries/'
MDA_PATH = 'sentiment_results/mda/top30/'
NEWS_PATH = 'sentiment_results/news/top30/'
REDDIT_PATH = 'sentiment_results/reddit/'

def load_price_data(ticker):
    """
    Load price data and calculate log returns
    """
    file_path = os.path.join(PRICE_PATH, f"{ticker}_prices.csv")
    logger.info(f"Loading price data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"Price data file not found: {file_path}")
        return None
    
    df = pd.read_csv(file_path)
    
    # Ensure Date column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Sort by date
    df = df.sort_values('Date')
    
    # Calculate log returns
    df['log_return'] = np.log(df['Adjusted_close'] / df['Adjusted_close'].shift(1))
    
    # Add target variable (next day's return)
    df['next_day_return'] = df['log_return'].shift(-1)
    
    logger.info(f"Loaded price data: {df.shape[0]} rows, {df.shape[1]} columns")
    return df


def load_reddit_sentiment(year_list=['2023', '2024']):
    """
    Load and aggregate Reddit sentiment data with volume weighting
    """
    all_data = []
    logger.info(f"Loading Reddit sentiment data for years: {year_list}")
    
    for year in year_list:
        file_path = os.path.join(REDDIT_PATH, f"reddit_sentiment_{year}_v3.csv")
        if os.path.exists(file_path):
            logger.info(f"Loading Reddit data from {file_path}")
            df = pd.read_csv(file_path)
            all_data.append(df)
        else:
            logger.warning(f"Reddit data file not found: {file_path}")
    
    if not all_data:
        logger.error("No Reddit data files found")
        return None
    
    # Concatenate all years of data
    reddit_data = pd.concat(all_data, ignore_index=True)
    
    # Select only the specified columns
    specified_columns = [
        'subreddit', 'date', 
        'polarity_detection', 'emotion_detection', 'intent_analysis', 'subjectivity_objectivity',
        'fine_grained_sentiment_avg', 'aspect_based_sentiment_avg', 'topic_sentiment_analysis_avg', 'contextual_sentiment_avg',
        'gemini_overall_sentiment', 'gemini_emotional_sentiment', 'gemini_contextual_sentiment',
        'vader_compound', 'vader_pos', 'vader_neg', 'vader_neu',
        'finbert_positive', 'finbert_negative', 'finbert_neutral', 'finbert_sentiment'
    ]
    
    available_columns = list(set(specified_columns) & set(reddit_data.columns))
    reddit_data = reddit_data[available_columns]
    
    # Convert date to datetime
    reddit_data['date'] = pd.to_datetime(reddit_data['date'])
    
    # Calculate post count per day
    daily_post_count = reddit_data.groupby('date').size().reset_index(name='post_count')
    
    # Calculate mean sentiment for each specified sentiment column
    sentiment_columns = [col for col in available_columns if col not in ['subreddit', 'date']]
    
    # Initialize an empty DataFrame to store results
    daily_sentiment = daily_post_count[['date', 'post_count']].copy()
    
    # Calculate log-weighted sentiment for each column
    for col in sentiment_columns:
        # Calculate mean sentiment per day
        mean_sentiment = reddit_data.groupby('date')[col].mean().reset_index(name=f'mean_{col}')
        
        # Merge with post count
        daily_sentiment = pd.merge(daily_sentiment, mean_sentiment, on='date', how='left')
        
        # Calculate volume-weighted sentiment: Mean Sentiment × log(1+Post Count)
        daily_sentiment[f'reddit_{col}'] = daily_sentiment[f'mean_{col}'] * np.log1p(daily_sentiment['post_count'])
        
        # Drop the mean column to keep the DataFrame clean
        daily_sentiment = daily_sentiment.drop(f'mean_{col}', axis=1)
    
    # Rename date column for consistency
    daily_sentiment = daily_sentiment.rename(columns={'date': 'Date'})
    
    logger.info(f"Processed Reddit sentiment data: {daily_sentiment.shape[0]} rows, {daily_sentiment.shape[1]} columns")
    return daily_sentiment


def load_news_sentiment(ticker):
    """
    Load and aggregate news sentiment data with volume weighting
    """
    file_path = os.path.join(NEWS_PATH, f"news_sentiment_{ticker}_with_vader_finbert.csv")
    logger.info(f"Loading news sentiment data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"News sentiment file not found: {file_path}")
        return None
    
    news_data = pd.read_csv(file_path)
    
    # Select only the specified columns
    specified_columns = [
        'ticker', 'date', 'title',  # Including title to count articles
        'polarity_detection', 'emotion_detection', 'intent_analysis', 'subjectivity_objectivity',
        'fine_grained_sentiment_avg', 'aspect_based_sentiment_avg', 'topic_sentiment_analysis_avg', 'contextual_sentiment_avg',
        'gemini_overall_sentiment', 'gemini_emotional_sentiment', 'gemini_contextual_sentiment',
        'vader_compound', 'vader_pos', 'vader_neg', 'vader_neu',
        'finbert_positive', 'finbert_negative', 'finbert_neutral', 'finbert_sentiment'
    ]
    
    available_columns = list(set(specified_columns) & set(news_data.columns))
    news_data = news_data[available_columns]
    
    # Convert date to datetime
    news_data['date'] = pd.to_datetime(news_data['date'])
    
    # Calculate article count per day
    daily_article_count = news_data.groupby('date').size().reset_index(name='article_count')
    
    # Calculate mean sentiment for each specified sentiment column
    sentiment_columns = [col for col in available_columns if col not in ['ticker', 'date', 'title']]
    
    # Initialize an empty DataFrame to store results
    daily_sentiment = daily_article_count[['date', 'article_count']].copy()
    
    # Calculate log-weighted sentiment for each column
    for col in sentiment_columns:
        # Calculate mean sentiment per day
        mean_sentiment = news_data.groupby('date')[col].mean().reset_index(name=f'mean_{col}')
        
        # Merge with article count
        daily_sentiment = pd.merge(daily_sentiment, mean_sentiment, on='date', how='left')
        
        # Calculate volume-weighted sentiment: Mean Sentiment × log(1+Article Count)
        daily_sentiment[f'news_{col}'] = daily_sentiment[f'mean_{col}'] * np.log1p(daily_sentiment['article_count'])
        
        # Drop the mean column to keep the DataFrame clean
        daily_sentiment = daily_sentiment.drop(f'mean_{col}', axis=1)
    
    # Rename date column for consistency
    daily_sentiment = daily_sentiment.rename(columns={'date': 'Date'})
    
    logger.info(f"Processed news sentiment data: {daily_sentiment.shape[0]} rows, {daily_sentiment.shape[1]} columns")
    return daily_sentiment


def load_mda_sentiment(ticker, alpha=0.2):
    """
    Load MDA sentiment data with dynamic exponential decay weighting
    
    For each daily data point, we calculate a weighted average of all previous
    quarterly reports, where the weight decays exponentially with the time distance.
    """
    file_path = os.path.join(MDA_PATH, f"{ticker}_2023_24.csv")
    logger.info(f"Loading MDA sentiment data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"MDA sentiment file not found: {file_path}")
        return None
    
    mda_data = pd.read_csv(file_path)
    
    # Select only the specified columns
    specified_columns = [
        'Date',
        'polarity_detection', 'emotion_detection', 'intent_analysis', 'subjectivity_objectivity',
        'gemini_overall_sentiment', 'gemini_emotional_sentiment', 'gemini_contextual_sentiment',
        'fine_grained_sentiment_avg', 'aspect_based_sentiment_avg', 'topic_sentiment_analysis_avg', 'contextual_sentiment_avg',
        'vader_compound', 'vader_pos', 'vader_neg', 'vader_neu',
        'finbert_positive', 'finbert_negative', 'finbert_neutral', 'finbert_sentiment'
    ]
    
    available_columns = list(set(specified_columns) & set(mda_data.columns))
    mda_data = mda_data[available_columns]
    
    # Convert date to datetime
    mda_data['Date'] = pd.to_datetime(mda_data['Date'])
    
    # Sort by date
    mda_data = mda_data.sort_values('Date')
    
    # Get sentiment columns
    sentiment_columns = [col for col in available_columns if col != 'Date']
    
    # Create a date range that spans the entire period (daily)
    start_date = mda_data['Date'].min()
    end_date = mda_data['Date'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create a DataFrame with all dates
    daily_df = pd.DataFrame({'Date': all_dates})
    
    # Calculate lambda from alpha
    # Relationship: α ≈ 1 - e^(-λ)
    # So λ ≈ -ln(1-α)
    lambda_param = -np.log(1 - alpha)
    logger.info(f"Using decay parameter lambda={lambda_param:.4f} (from alpha={alpha})")
    
    # For each sentiment column, calculate weighted values for all days
    weighted_columns = {}
    
    for col in sentiment_columns:
        # Create a Series for this column
        col_values = pd.Series(index=mda_data['Date'], data=mda_data[col].values)
        
        # For each day, calculate weighted average of all previous MDA reports
        weighted_values = []
        
        for day in all_dates:
            # Get all MDA reports up to this day
            past_reports = col_values[col_values.index <= day]
            
            if past_reports.empty:
                weighted_values.append(np.nan)
                continue
            
            # Calculate days since each report
            days_since = [(day - report_date).days for report_date in past_reports.index]
            
            # Calculate weights using continuous exponential decay
            weights = np.exp(-lambda_param * np.array(days_since))
            
            # Normalize weights to sum to 1
            weights = weights / weights.sum()
            
            # Calculate weighted average
            weighted_avg = np.sum(past_reports.values * weights)
            weighted_values.append(weighted_avg)
        
        # Add to dictionary of weighted columns
        weighted_columns[f'mda_{col}'] = weighted_values
    
    # Add weighted columns to daily DataFrame
    for col_name, values in weighted_columns.items():
        daily_df[col_name] = values
    
    logger.info(f"Processed MDA sentiment data with dynamic decay: {daily_df.shape[0]} rows, {daily_df.shape[1]} columns")
    return daily_df


def load_roe_pe_data(ticker, alpha=0.2):
    """
    Load and process ROE/PE data with dynamic exponential decay weighting
    
    For each daily data point, we calculate a weighted average of all previous
    quarterly reports, where the weight decays exponentially with the time distance.
    """
    file_path = os.path.join(ROE_PE_PATH, f"{ticker}_2023_24.csv")
    logger.info(f"Loading ROE/PE data from {file_path}")
    
    if not os.path.exists(file_path):
        logger.error(f"ROE/PE file not found: {file_path}")
        return None
    
    roe_pe_data = pd.read_csv(file_path)
    
    # Convert date to datetime
    roe_pe_data['Date'] = pd.to_datetime(roe_pe_data['Date'])
    
    # Sort by date
    roe_pe_data = roe_pe_data.sort_values('Date')
    
    # Convert ROE from string percentage to float if needed
    if roe_pe_data['ROE'].dtype == 'object':
        roe_pe_data['ROE'] = roe_pe_data['ROE'].str.replace('%', '').astype(float) / 100
    
    # Create a date range that spans the entire period (daily)
    start_date = roe_pe_data['Date'].min()
    end_date = roe_pe_data['Date'].max()
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Create a DataFrame with all dates
    daily_df = pd.DataFrame({'Date': all_dates})
    
    # Calculate lambda from alpha
    # Relationship: α ≈ 1 - e^(-λ)
    # So λ ≈ -ln(1-α)
    lambda_param = -np.log(1 - alpha)
    logger.info(f"Using decay parameter lambda={lambda_param:.4f} (from alpha={alpha})")
    
    # For each financial metric, calculate weighted values for all days
    weighted_columns = {}
    original_columns = {}
    
    for col in ['PE_Ratio', 'ROE']:
        # Create a Series for this column
        col_values = pd.Series(index=roe_pe_data['Date'], data=roe_pe_data[col].values)
        
        # Keep track of original values for each date (for comparison)
        original_series = pd.Series(index=roe_pe_data['Date'], data=roe_pe_data[col].values)
        original_df = pd.DataFrame({'Date': original_series.index, col: original_series.values})
        
        # For each day, calculate weighted average of all previous financial reports
        weighted_values = []
        
        for day in all_dates:
            # Get all financial reports up to this day
            past_reports = col_values[col_values.index <= day]
            
            if past_reports.empty:
                weighted_values.append(np.nan)
                continue
            
            # Calculate days since each report
            days_since = [(day - report_date).days for report_date in past_reports.index]
            
            # Calculate weights using continuous exponential decay
            weights = np.exp(-lambda_param * np.array(days_since))
            
            # Normalize weights to sum to 1
            weights = weights / weights.sum()
            
            # Calculate weighted average
            weighted_avg = np.sum(past_reports.values * weights)
            weighted_values.append(weighted_avg)
        
        # Add to dictionaries
        weighted_columns[f'weighted_{col}'] = weighted_values
        
        # Merge original values
        daily_df = pd.merge(daily_df, original_df, on='Date', how='left')
    
    # Add weighted columns to daily DataFrame
    for col_name, values in weighted_columns.items():
        daily_df[col_name] = values
    
    # Forward fill missing original values
    daily_df = daily_df.fillna(method='ffill')
    
    logger.info(f"Processed ROE/PE data with dynamic decay: {daily_df.shape[0]} rows, {daily_df.shape[1]} columns")
    return daily_df


def merge_datasets(price_df, reddit_df=None, news_df=None, mda_df=None, roe_pe_df=None):
    """
    Merge all datasets on Date
    """
    # Start with price data as the base
    merged_df = price_df.copy()
    logger.info(f"Starting merge with price data: {merged_df.shape[0]} rows")
    
    # Merge Reddit sentiment
    if reddit_df is not None:
        merged_df = pd.merge(merged_df, reddit_df, on='Date', how='left')
        logger.info(f"After merging Reddit data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    # Merge News sentiment
    if news_df is not None:
        merged_df = pd.merge(merged_df, news_df, on='Date', how='left')
        logger.info(f"After merging News data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    # Merge MDA with dynamic decay weighting
    if mda_df is not None:
        # With the new dynamic decay approach, we now have daily values
        # so we can do a simple left join without forward fill
        merged_df = pd.merge(merged_df, mda_df, on='Date', how='left')
        logger.info(f"After merging MDA data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    # Merge ROE/PE with dynamic decay weighting
    if roe_pe_df is not None:
        # With the new dynamic decay approach, we now have daily values
        # so we can do a simple left join without forward fill
        merged_df = pd.merge(merged_df, roe_pe_df, on='Date', how='left')
        logger.info(f"After merging ROE/PE data: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
    
    return merged_df


def process_ticker(ticker, alpha_mda=0.2, alpha_roe_pe=0.2, reddit_years=['2023', '2024']):
    """
    Process all data for a single ticker
    """
    logger.info(f"Starting data processing for {ticker}")
    
    # 1. Load all datasets
    price_data = load_price_data(ticker)
    if price_data is None:
        logger.error(f"Failed to load price data for {ticker}")
        return None
    
    reddit_data = load_reddit_sentiment(reddit_years)
    news_data = load_news_sentiment(ticker)
    mda_data = load_mda_sentiment(ticker, alpha=alpha_mda)
    roe_pe_data = load_roe_pe_data(ticker, alpha=alpha_roe_pe)
    
    # 2. Merge datasets
    merged_data = merge_datasets(price_data, reddit_data, news_data, mda_data, roe_pe_data)
    
    # 3. Filter to only include data from 2022-01-01 to 2024-12-31
    start_date = pd.Timestamp('2022-01-01')
    end_date = pd.Timestamp('2024-12-31')
    merged_data = merged_data[(merged_data['Date'] >= start_date) & (merged_data['Date'] <= end_date)]
    logger.info(f"Filtered data to date range: {start_date} to {end_date}")
    
    # 4. Final preprocessing
    # Replace infinities with NaN
    merged_data = merged_data.replace([np.inf, -np.inf], np.nan)
    
    # Drop rows with NaN in log_return or next_day_return
    # These are essential as they are the target variables
    merged_data = merged_data.dropna(subset=['log_return', 'next_day_return'])
    
    # For other columns, fill NaNs with column means
    cols_to_fill = [col for col in merged_data.columns if col not in ['Date', 'log_return', 'next_day_return']]
    merged_data[cols_to_fill] = merged_data[cols_to_fill].fillna(merged_data[cols_to_fill].mean())
    
    logger.info(f"Final dataset for {ticker} has shape: {merged_data.shape}")
    
    # Create a column with the ticker name for reference
    merged_data['ticker'] = ticker
    
    return merged_data


def create_dataset_variants(merged_data):
    """
    Create different variants of the dataset for model comparison:
    1. Price data only
    2. Price + PE/ROE data only
    3. Price + Sentiment data only
    4. All data
    """
    # Price data only (keep necessary columns)
    price_cols = ['Date', 'ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'log_return', 'next_day_return']
    price_only = merged_data[price_cols].copy()
    
    # Price + PE/ROE
    pe_roe_cols = [col for col in merged_data.columns if any(term in col for term in ['PE_Ratio', 'ROE'])]
    price_pe_roe = merged_data[price_cols + pe_roe_cols].copy()
    
    # Price + Sentiment
    sentiment_cols = [col for col in merged_data.columns if any(
        prefix in col for prefix in ['reddit_', 'news_', 'mda_'])]
    price_sentiment = merged_data[price_cols + sentiment_cols].copy()
    
    # All data is already in merged_data
    
    return {
        'price_only': price_only,
        'price_pe_roe': price_pe_roe,
        'price_sentiment': price_sentiment,
        'all_data': merged_data
    }


def save_processed_data(dataset_variants, ticker, output_dir='processed_data'):
    """
    Save all dataset variants to files
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save each variant
    for variant_name, data in dataset_variants.items():
        file_path = os.path.join(output_dir, f"{ticker}_{variant_name}.csv")
        data.to_csv(file_path, index=False)
        logger.info(f"Saved {variant_name} dataset to {file_path}")


def main():
    """
    Main function to process multiple tickers
    """
    # List of tickers to process
    tickers = ['AAPL']  # Replace with your list of tickers
    
    for ticker in tickers:
        # Process the ticker
        merged_data = process_ticker(ticker)
        
        if merged_data is not None:
            # Create dataset variants
            dataset_variants = create_dataset_variants(merged_data)
            
            # Save processed data
            save_processed_data(dataset_variants, ticker)
            
            logger.info(f"Completed processing for {ticker}")
        else:
            logger.error(f"Failed to process {ticker}")


if __name__ == "__main__":
    main()

2025-04-27 16:34:48,157 - stock_data_processing - INFO - Starting data processing for AAPL
2025-04-27 16:34:48,158 - stock_data_processing - INFO - Loading price data from data/stock_price_data/top30/AAPL_prices.csv
2025-04-27 16:34:48,164 - stock_data_processing - INFO - Loaded price data: 1561 rows, 9 columns
2025-04-27 16:34:48,164 - stock_data_processing - INFO - Loading Reddit sentiment data for years: ['2023', '2024']
2025-04-27 16:34:48,165 - stock_data_processing - INFO - Loading Reddit data from sentiment_results/reddit/reddit_sentiment_2023_v3.csv
2025-04-27 16:34:48,265 - stock_data_processing - INFO - Loading Reddit data from sentiment_results/reddit/reddit_sentiment_2024_v3.csv
2025-04-27 16:34:48,440 - stock_data_processing - INFO - Processed Reddit sentiment data: 704 rows, 21 columns
2025-04-27 16:34:48,443 - stock_data_processing - INFO - Loading news sentiment data from sentiment_results/news/top30/news_sentiment_AAPL_with_vader_finbert.csv
2025-04-27 16:34:48,529 - s